# Neo4j Hello World (Notebook) - Friends Use Case

This notebook connects to a local Neo4j **Community** instance (via Docker), creates a tiny graph, and queries it.

**Assumes** 
 
 
- Neo4j service is running at `bolt://localhost:${URI_PORT}` with the user and password set in the `.env` file. **Run `docker compose up -d`**.
- Ollama service is up on `http://localhost:11434` (ollama default). **Run `ollama serve` and pull the model `ollama pull nomic-embed-text`** (if not pulled yet).



In [1]:
import os
from dotenv import load_dotenv  
import yaml
from pathlib import Path
from pprint import pprint
from termcolor import cprint

from langchain_neo4j import Neo4jGraph


In [2]:
load_dotenv()  # Load local environment variables

URI = "bolt://localhost:" + os.environ.get("URI_PORT")
NEO4J_USER = os.environ.get("NEO4J_USER")
NEO4J_PWD = os.environ.get("NEO4J_PASSWORD")
NEO4J_DB = os.getenv("NEO4J_DATABASE", "neo4j")    # 👈 choose DB here
EMBED_MODEL = "nomic-embed-text:latest"

cprint(f"Connecting to Neo4j at {URI} with user {NEO4J_USER} and password {NEO4J_PWD}", "green")

Connecting to Neo4j at bolt://localhost:7687 with user neo4j and password test1234


In [3]:
# load cypher queries from yaml file
queries = yaml.safe_load(Path("queries_friends.yaml").read_text())
queries.keys()  # list available queries

dict_keys(['constraints', 'create_seed', 'show_people', 'show_companies', 'match_adjacency', 'show_info', 'add_info', 'create_vector_indexes', 'delete_all'])

## Using Langchain wrapper for Neo4j

In [4]:
kg = Neo4jGraph(url=URI, username=NEO4J_USER, password=NEO4J_PWD, database=NEO4J_DB)

### 1. Create sample data

<p align="center">
  <img src="media/KG_step1_populate_graph.svg" width="550">
</p>


- **Entities**: Person and Company nodes with unique constraints (unique name and uuid)
- **Relationships**: KNOWS (person-to-person) and WORKS_AT (person-to-company)
- **Properties**: Basic attributes (name, age, education, industry)


In [5]:
# KG

wipe_at_init = True # delete everything at the start 

# Interact directly with KG, no need for driver context.

cprint(f"\n== Connected to Neo4j database: {NEO4J_DB}", "green")

cprint("\n== Creating constraints (if not exist)", "green")
for q in queries["constraints"]:
    kg.query(q)
print(" ok")

cprint("\n== Init Cleanup.", "green")
if wipe_at_init:
    for q in queries["delete_all"]:
        kg.query(q)
    print(" ok")
else:
    print(" skipped")
    
cprint("\n== Creating sample data", "green")
kg.query(queries["create_seed"])
print(" ok")

cprint("\n== Query: list all people", "green")
records = kg.query(queries["show_people"]) # <class 'list'>
for r in records:
    print(r)
    
cprint("\n== Query: list all companies", "green")
records = kg.query(queries["show_companies"]) # <class 'list'>
for r in records:
    print(r)

cprint("\n== Query: adjacency (who knows whom)", "green")
records = kg.query(queries["match_adjacency"]) # <class 'list'>
for r in records:
    print(r)


== Connected to Neo4j database: neo4j

== Creating constraints (if not exist)
 ok

== Init Cleanup.
 ok

== Creating sample data
 ok

== Query: list all people
{'name': 'Paula', 'age': 25, 'p.gender': 'female', 'education': 'Computer Engineering'}
{'name': 'Guillermo', 'age': 26, 'p.gender': 'male', 'education': 'Industrial Engineering'}
{'name': 'Gabriela', 'age': 26, 'p.gender': 'female', 'education': 'Physics'}
{'name': 'Iria', 'age': 27, 'p.gender': 'female', 'education': 'Physics'}
{'name': 'Cristina', 'age': 27, 'p.gender': 'female', 'education': 'Physics'}

== Query: list all companies
{'name': 'Indra', 'industry': 'Engineering'}
{'name': 'CIEMAT', 'industry': 'Scientific Research'}
{'name': 'CBM', 'industry': 'Scientific Research'}

== Query: adjacency (who knows whom)
{'person': 'Cristina', 'knows': ['Gabriela', 'Iria'], 'works_at': 'CBM'}
{'person': 'Gabriela', 'knows': ['Cristina', 'Iria'], 'works_at': 'CIEMAT'}
{'person': 'Guillermo', 'knows': ['Paula', 'Iria'], 'works_at'

### 2. Add rich text info

<p align="center">
  <img src="media/KG_step2_generate_rich_descriptions.svg" width="750">
</p>


In [6]:
cprint("\n== Query: Adding descriptions, appearance and summaries", "green")
for q in queries["add_info"]:
    kg.query(q)
print(" ok")

records = kg.query(queries["show_info"])
for r in records:
    print(r)



== Query: Adding descriptions, appearance and summaries
 ok
{'labels(n)': ['Person'], 'n.name': 'Iria', 'n.info': 'Iria is a female of 27 years old and studied Physics.Iria has blue eyes and long brunette and wavy hair. She likes to paint her nails in red or purple colours. She usually wears long earrings.'}
{'labels(n)': ['Person'], 'n.name': 'Guillermo', 'n.info': 'Guillermo is a male of 26 years old and studied Industrial Engineering.Guillermo has brown eyes and short hair. He has a very fancy shirt that he takes to all important events. He shaved his head this summer.'}
{'labels(n)': ['Person'], 'n.name': 'Gabriela', 'n.info': "Gabriela is a female of 26 years old and studied Physics.Gabriela has long curly hair with babylights. She's petite and likes to wear hippie-style clothes."}
{'labels(n)': ['Person'], 'n.name': 'Paula', 'n.info': 'Paula is a female of 25 years old and studied Computer Engineering.Paula short hair in a wolfcut style. She wears long and wide pants and sneaker

### 3. Create property embeddings (first step into RAG) 

<p align="center">
  <img src="media/KG_step3_generate_property_embeddings.svg" width="750">
</p>

**RAG** implementation requires selecting a **property to embed and use for similarity searches**. 

Description properties containing **rich text** work well for this purpose, as they provide richer semantic information. In our example, we'll use *info* and *info*.

In order to do so, we create two vector indexes in Neo4j:

- **Vector index *person_node_info_idx***: based on property ***info_emb*** for nodes of type "Person"
-  **Vector index *company_node_info_idx***: based on property ***info_emb*** for nodes of type "Company"

After that, we create the embeddings (this happens for both Person nodes and Company nodes):

- **Property *info*** ---`nomic-embed-text`---> **Property *info_emb***


In [7]:
# Create vector indexes (once)

for q in queries["create_vector_indexes"]:
    kg.query(q)

# Show created vector indexes
results = kg.query("SHOW VECTOR INDEXES")
idx = list(results)
cprint(f"\nFound {len(idx)} vector index entries.", "green")
for r in idx:
    cprint("-"*20,"green")
    pprint(r)


Found 3 vector index entries.
--------------------
{'entityType': 'NODE',
 'id': 18,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['Company'],
 'lastRead': None,
 'name': 'company_node_info_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['embedding'],
 'readCount': None,
 'state': 'ONLINE',
 'type': 'VECTOR'}
--------------------
{'entityType': 'RELATIONSHIP',
 'id': 15,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['KNOWS'],
 'lastRead': None,
 'name': 'knows_relationship_info_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['embedding'],
 'readCount': None,
 'state': 'ONLINE',
 'type': 'VECTOR'}
--------------------
{'entityType': 'NODE',
 'id': 5,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['Person'],
 'lastRead': None,
 'name': 'person_node_info_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['embedding'],
 'readCount': None,
 'state': 'ONLINE',
 'type': 'VECTOR'}


In [8]:
from helper_neo4j import vectorize_property

# (p:PERSON): create embeddings only for nodes missing them
vectorize_property("node", "Person", "info", kg=kg)

# (c:COMPANY): create embeddings only for nodes missing them
vectorize_property("node", "Company", "info", kg=kg)

# [r:KNOWS]: create embeddings only for nodes missing them
vectorize_property("relationship", "KNOWS", "info", kg=kg)


Generating embeddings for (n:Person) on n.info
  text: Iria is a female of 27 years old and studied Physics.Iria has blue eyes and long brunette and wavy hair. She likes to paint her nails in red or purple colours. She usually wears long earrings.
  vec: [0.031421136, 0.032104157, -0.15766615]
  text: Guillermo is a male of 26 years old and studied Industrial Engineering.Guillermo has brown eyes and short hair. He has a very fancy shirt that he takes to all important events. He shaved his head this summer.
  vec: [-0.023262369, 0.05322417, -0.15030223]
  text: Gabriela is a female of 26 years old and studied Physics.Gabriela has long curly hair with babylights. She's petite and likes to wear hippie-style clothes.
  vec: [0.028215451, 0.02993047, -0.18000375]
  text: Paula is a female of 25 years old and studied Computer Engineering.Paula short hair in a wolfcut style. She wears long and wide pants and sneakers to the laboratory.
  vec: [0.011417965, 0.040603343, -0.16852917]
  text: C

### 4. Search 

Whenever we query this graph, we can use two different but complementary search techniques:

1. **KG Retreival**: through **Neo4J Cypher Query Language (CQL)** we can query precise entities and relations. The input query must be translated into CQL to get the desired results.

2. **Vector Retrieval**: **embedding the input query**, we can make a vector search against the vector indexes defined above.

The results will be a combination of both searches.

<p align="center">
  <img src="media/KGRAG_schema.svg">
</p>


In [9]:
# Query Nodes
from helper_neo4j import neo4j_vector_search


# Query Nodes
result = neo4j_vector_search(element = "node",
                             question = "Who shaved its head this summer?", 
                             index = "person_node_info_idx",
                             kg=kg
                             )
for r in result:
        print(dict(r))


result  = neo4j_vector_search(element = "node",
                              question = "Which company investigates Cancer?",
                              index = "company_node_info_idx",
                              kg=kg
                              )
for r in result:
    print(dict(r))

# Query Relationships
result  = neo4j_vector_search(element = "relationship",
                              question = "Who is helping Iria at work?",
                              index = "knows_relationship_info_idx",
                              kg=kg
                              )
for r in result:
    print(dict(r))


Generating embeddings for question 'Who shaved its head this summer?'
  text: Who shaved its head this summer?
  vec: [0.022573026, -0.015476549, -0.17212495, 0.0018683294, -0.038688686, 0.044817436, 0.01842398, 0.013745231, 0.050375726, 0.008345599]

{'score': 0.8374781608581543, 'node': {'name': 'Guillermo', 'age': 26, 'gender': 'male', 'uuid': '9392285d-9850-4f5c-929d-c02a34fe2491', 'embedding': '', 'education': 'Industrial Engineering', 'info': 'Guillermo is a male of 26 years old and studied Industrial Engineering.Guillermo has brown eyes and short hair. He has a very fancy shirt that he takes to all important events. He shaved his head this summer.'}}
{'score': 0.7817301750183105, 'node': {'name': 'Gabriela', 'age': 26, 'gender': 'female', 'uuid': '6d70197b-bb99-4de1-8dfd-7f13c8d01d5a', 'embedding': '', 'education': 'Physics', 'info': "Gabriela is a female of 26 years old and studied Physics.Gabriela has long curly hair with babylights. She's petite and likes to wear hippie-styl